# Use Obspy to manage inventory XML files

## Fei Zhang
## created on 2020-09-18


- modify station xml to inlcude new GA-metadata

- split a big xml file into multiple single-station xml files

- concatenate the mutiple station xml files into one big xml file

In [1]:
import os
import obspy
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL


In [2]:
print (obspy.__version__)

1.2.1


In [3]:
METADB_DIR = "/g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata"

ORIG_INVENTORY_FILE= os.path.join(METADB_DIR, "SrcInventoryXML", "OA_stations_2017-2018.xml" )
# "/Datasets/InventoryXml/OA_stations_2017-2018.xml"

#output 
OUTPUT_DIR= os.path.join(METADB_DIR, "Output_Dir")  # "/Datasets/InventoryXml/OA_stations_2017-2018"



In [4]:
if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [5]:
big_inv = obspy.read_inventory(ORIG_INVENTORY_FILE) 


In [6]:
os.path.basename(ORIG_INVENTORY_FILE)

'OA_stations_2017-2018.xml'

In [7]:
filename_new_xml= os.path.basename(ORIG_INVENTORY_FILE).replace(".xml", "_new.xml")

new_big_xml =os.path.join(OUTPUT_DIR, filename_new_xml)

print("New XML file = ", new_big_xml)


New XML file =  /g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata/Output_Dir/OA_stations_2017-2018_new.xml


In [8]:
# Write the inventory into a new file to see if they are identical?

big_inv.write(new_big_xml, format="stationxml", validate=True)

# 
# Problem:  Missing a lot of startDate for some stations grep "<Station" OA_stations2.xml 

#  $ grep "<Station" OA_stations_2017-2018_new.xml 
#     <Station code="BS24" startDate="2017-09-27T05:16:47.000000Z" endDate="2018-07-04T04:38:50.000000Z">
#     <Station code="BS25" endDate="2018-06-27T11:44:37.000000Z">
#     <Station code="BS26" startDate="2017-09-27T00:49:10.000000Z" endDate="2018-01-12T04:02:13.000000Z">
#     <Station code="BS27" startDate="2017-09-26T00:42:49.000000Z" endDate="2018-07-03T03:04:17.000000Z">
#     <Station code="BS28" startDate="2017-09-26T02:02:42.000000Z" endDate="2018-07-03T02:37:29.000000Z">
#     <Station code="BT23" startDate="2017-09-28T04:11:41.000000Z" endDate="2018-07-04T03:40:38.000000Z">
#     <Station code="BT24" startDate="2017-09-27T06:14:32.000000Z" endDate="2018-06-26T22:36:58.000000Z">
#     <Station code="BT25" startDate="2017-09-27T02:45:22.000000Z" endDate="2018-05-19T11:23:10.000000Z">
#     <Station code="BT26" startDate="2017-09-26T23:41:45.000000Z" endDate="2017-10-27T00:19:21.000000Z">


In [9]:
type(big_inv.networks)

list

In [10]:
for net in big_inv.networks:
    print ("The Network Code", net.code)
    number_of_stations = len(net.stations)
    print("The total number of stations/nodes =", number_of_stations )
    
    for i in range(number_of_stations):
        print(net.stations[i].code, net.stations[i].start_date, net.stations[i].end_date)
        
        # issue: why the obspy read_invnetory() missed some start_date for stations?

The Network Code OA
The total number of stations/nodes = 328
BS24 2017-09-27T05:16:47.000000Z 2018-07-04T04:38:50.000000Z
BS25 None 2018-06-27T11:44:37.000000Z
BS26 2017-09-27T00:49:10.000000Z 2018-01-12T04:02:13.000000Z
BS27 2017-09-26T00:42:49.000000Z 2018-07-03T03:04:17.000000Z
BS28 2017-09-26T02:02:42.000000Z 2018-07-03T02:37:29.000000Z
BT23 2017-09-28T04:11:41.000000Z 2018-07-04T03:40:38.000000Z
BT24 2017-09-27T06:14:32.000000Z 2018-06-26T22:36:58.000000Z
BT25 2017-09-27T02:45:22.000000Z 2018-05-19T11:23:10.000000Z
BT26 2017-09-26T23:41:45.000000Z 2017-10-27T00:19:21.000000Z
BT27 2017-09-25T23:24:09.000000Z 2018-06-30T15:52:25.000000Z
BT28 2017-10-04T11:28:39.000000Z 2018-07-03T02:08:32.000000Z
BU22 2017-09-28T23:56:41.000000Z 2018-07-04T01:42:50.000000Z
BU23 2017-09-28T22:44:17.000000Z 2018-07-04T03:12:26.000000Z
BU24 2017-09-28T05:11:40.000000Z 2018-07-03T23:16:10.000000Z
BU25 2017-09-25T22:35:23.000000Z 2018-06-19T05:05:57.000000Z
BU26 2017-09-26T22:31:06.000000Z 2018-07-02T04:

In [11]:
# Split the big inventory station XML file into multiple one-station.xml file, according the station CODE

for a_net in big_inv.networks:
    for a_sta in a_net.stations:
        
        print(a_net.code, a_sta.code)  # This print 328 (OA, sta) pairs - they are NOT unique!!!
        
        a_inv = big_inv.select(network=a_net.code, station=a_sta.code)  #.copy()
        
        # modify station metadata
#         my_sensor=obspy.core.inventory.util.Equipment(type="Sensor", description="Nanometrics Trillium Compact 120s",serial_number="004940")
#         my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description="Guralp Minimus",serial_number="MIN-A456")
#         a_sta.equipments = [my_sensor, my_digitizer]
        
        sta_file_name = "%s_%s_station.xml"%(a_net.code, a_sta.code)
        
        outxml = os.path.join(OUTPUT_DIR, sta_file_name)
        a_inv.write(outxml,format="stationxml", validate=True) # nsmap={'GeoscienceAustralia': GA_NameSpace})
        # 119 OA_*_station.xml file written. Some files written 3 times!!
# fzhang@zubuntu1804 /Datasets/InventoryXml/OA_stations_2017-2018 
#  $ ls *station.xml| wc
#     119     119    2380


    

OA BS24
OA BS25
OA BS26
OA BS27
OA BS28
OA BT23
OA BT24
OA BT25
OA BT26
OA BT27
OA BT28
OA BU22
OA BU23
OA BU24
OA BU25
OA BU26
OA BU27
OA BU28
OA BV21
OA BV22
OA BV23
OA BV24
OA BV26
OA BV27
OA BV28
OA BW20
OA BW21
OA BW22
OA BW23
OA BW24
OA BW25
OA BW26
OA BW27
OA BW28
OA BX20
OA BX21
OA BX22
OA BX23
OA BX24
OA BX25
OA BX26
OA BX27
OA BX28
OA BY20
OA BY21
OA BY22
OA BY23
OA BY24
OA BY25
OA BY26
OA BY27
OA BY28
OA BZ20
OA BZ21
OA BZ22
OA BZ23
OA BZ24
OA BZ25
OA BZ26
OA BZ27
OA BZ28
OA CA20
OA CA21
OA CA22
OA CA23
OA CA24
OA CA25
OA CA26
OA CA27
OA CA28
OA CB20
OA CB21
OA CB22
OA CB23
OA CB24
OA CB25
OA CB26
OA CB27
OA CB28
OA CC20
OA CC21
OA CC22
OA CC23
OA CC24
OA CC25
OA CC26
OA CC27
OA CC28
OA CD21
OA CD22
OA CD23
OA CD24
OA CD25
OA CD26
OA CD27
OA CD28
OA CE22
OA CE23
OA CE24
OA CE25
OA CE26
OA CE27
OA CE28
OA CF22
OA CF23
OA CF24
OA CF25
OA CF27
OA CF28
OA CG21
OA CG22
OA CG23
OA CH21
OA CH22
OA CH23
OA CI21
OA CI22
OA CI23
OA CJ27
OA BS24
OA BS25
OA BS27
OA BS28
OA BT23
OA BT24


In [12]:
# Construct a new inventory object of networks.
# This will use new obspy version and new attributes for Inventory

filename_newv_xml= os.path.basename(ORIG_INVENTORY_FILE).replace(".xml", "_new_version.xml")

new_version_xml =os.path.join(OUTPUT_DIR, filename_newv_xml)

print("New XML file = ", new_version_xml)

inv2 = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="Geoscience Australia EFTF AusArray")

for a_net in big_inv.networks:
    # Re-write each network of the big inventory into the new object inv2 
    # inv2.networks = []  
    inv2.networks.append(a_net)
    inv2.write(new_version_xml, format="stationxml", validate=True)  # every Station got equipment
    
    print("New Version XML file = ", new_version_xml, len(a_net.stations))

New XML file =  /g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata/Output_Dir/OA_stations_2017-2018_new_version.xml
New Version XML file =  /g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata/Output_Dir/OA_stations_2017-2018_new_version.xml 328


# Modify the existing station XML files inclding new metadata

- add sensor digitizer
- add extra metadata: GPS correction
- add extra metadata: Orientation correction

In [13]:
# Construct a new inventory object of networks.
# This will use new obspy version and new attributes:
inv2 = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="Geoscience Australia EFTF AusArray PST")


In [14]:
 GA_NameSpace = "https://github.com/GeoscienceAustralia/hiperseis"

# Original station inventory XML file to be modified
# /g/data/ha3/Passive/_AusArray/OA/ASDF_cleaned/OA_stations_2017-2018.xml
#in_station_xml_file = "/Datasets/StationXML_with_time_corrections2/OA.CF28_station_inv_modified.xml"

# extra metadata info file(s) to be read and formatted into JSON
in_csv_file = os.path.join(METADB_DIR, "OA_All_GPSClock_Correction.csv")
#"/Datasets/GPS_ClockCorr/OA.CE28_clock_correction.csv" # "./OA.CF28_clock_correction.csv"

in_json_file = os.path.join(METADB_DIR, "OrientationCorr","OA_ori_error_estimates.json")
# "/Datasets/Orientation_Correction_json/OA_ori_error_estimates.json"
# output dir for modified station inventory xml files
out_dir = OUTPUT_DIR
#"/home/fzhang/tmpdir"
    
import sys
sys.path.append("/home/fzhang/Githubz/hiperseis")
sys.path.append("/g/data/ha3/fxz547/Githubz/hiperseis/")

import json
import obspy
from obspy.core.util import AttribDict
from obspy.core import UTCDateTime

from seismic.inventory.extract_equipments_from_csv import  EquipmentExtractor

from seismic.inventory.station_metadata_extra import StationMetadataExtra,get_csv_correction_data, get_orientation_corr

net, sta, csv_data = get_csv_correction_data(in_csv_file)
net_sta, oricorr_json_data = get_orientation_corr(in_json_file)


my_equip_obj = EquipmentExtractor(csvfile=os.path.join(METADB_DIR,"Equipments/FieldSiteVisitLive.csv"))

for a_net in big_inv.networks:
    
    print("The number of station-nodes in the network =", len(a_net.stations))
    
    for a_sta in a_net.stations:
        # print(a_net.code, a_sta.code)  # this contains 328 pairs, but they are NOT unique, station code may repeat.

        
        a_inv = big_inv.select(network=a_net.code,station=a_sta.code)  # .copy appears to have no effect here
        
        # print (a_sta.code, " stations has %s channels"%len(a_sta))
        
        _sensors = my_equip_obj.get_sensors(a_net.code, a_sta.code)
        if len(_sensors)>0:
            sensor_desc =_sensors[0].get("Description")
            sensor_sernumb =_sensors[0].get("SerNumber")
        else:
            print("%s %s  No sensors !"%(a_net.code, a_sta.code))
            sensor_desc = "NA Sensor for (%s,%s)"%(a_net.code, a_sta.code)
            sensor_sernumb = "NA"
            
        _digitizers = my_equip_obj.get_digitizer(a_net.code, a_sta.code)
        if len(_digitizers)>0:
            dig_desc =_digitizers[0].get("Description")
            dig_sernumb = _digitizers[0].get("SerNumber")
        else:
            print("%s %s  No digitizers !"%(a_net.code, a_sta.code))
            dig_desc = "NA Digitizer for (%s,%s)"%(a_net.code, a_sta.code)
            dig_sernumb = "NA"
         
        
        # modify station metadata
        my_sensor=obspy.core.inventory.util.Equipment(type="Sensor", description=sensor_desc,serial_number=sensor_sernumb)
        
        #my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description="Guralp Minimus",serial_number="MIN-A456")
        my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description=dig_desc,serial_number=dig_sernumb)

        a_sta.equipments = [my_sensor, my_digitizer]
        
        # get station start_ end_date and split csv_data
        start_dt = a_sta.start_date
        end_dt = a_sta.end_date

        ajson = StationMetadataExtra(a_net.code, a_sta.code, start_datetime=start_dt, end_datetime=end_dt)

        # generate/format extra metadata from inputs
        mpdf = ajson.add_gps_correction_from_csv(csv_data)

        # updated the ajson object with more metadata, such as orientation corr
        ajson.add_orientation_correction(oricorr_json_data)

        ajson.write_metadata2json(
            os.path.join(out_dir, "%s.%s_%s_extra_metadata.json" % (a_net.code, a_sta.code, str(start_dt))))

        # Now, ready to write the ajson obj into new xml file
        mformat = "JSON"

        my_tag = AttribDict()
        my_tag.namespace = GA_NameSpace

        my_tag.value = ajson.make_json_string()  # store all the extra metadata into a json string.

        a_sta.extra = AttribDict()
        a_sta.extra.GAMetadata = my_tag

        # prepare to write out a modified xml file
        stationxml_with_extra = '%s.%s_station_metadata_%s.xml' % (a_net.code, a_sta.code, mformat)

        if out_dir is not None and os.path.isdir(out_dir):
            stationxml_with_extra = os.path.join(out_dir, stationxml_with_extra)

        a_inv.write(stationxml_with_extra, format='STATIONXML',
                           nsmap={'GeoscienceAustralia': GA_NameSpace})

#         sta_file_name2 = "%s_%s_station2.xml"%(a_net.code, a_sta.code)  
#         # OA_CE28 was written 3-times!!!!!! due to multiple (OA,CE28)-station-nodes 
#         There will be 119 xml files written in this loop of 328 items. However, the final results missed 119 equipments!!
#         outxml2 = os.path.join(OUTPUT_DIR, sta_file_name2)

#         inv2.networks = a_inv.networks

#         inv2.write(outxml2,format="stationxml", validate=True) # nsmap={'GeoscienceAustralia': GA_NameSpace})

    # After the modification of ALL the station objects, 
    # write the big inventory in new object inv2 
    inv2.networks = []  
    inv2.networks.append(a_net)
    inv2.write(a_net.code+"_stations2.xml", format="stationxml", nsmap={'GeoscienceAustralia': GA_NameSpace}, validate=True)  # every Station got equipment
    
    # and the original write out again to check if it has been modified 
    big_inv.write(a_net.code+"_stations_post.xml", format="stationxml", nsmap={'GeoscienceAustralia': GA_NameSpace}, validate=True) # also has the Sensors etc

The number of station-nodes in the network = 328
OA BS24  No sensors !
OA BS24  No digitizers !
OA BS24  GPS corrections =  0
OA BS25  No sensors !
OA BS25  No digitizers !
OA BS25  GPS corrections =  0
OA BS26  No sensors !
OA BS26  No digitizers !
OA BS26  GPS corrections =  0
OA BS27  No sensors !
OA BS27  GPS corrections =  0
OA BS28  No sensors !
OA BS28  GPS corrections =  0
OA BT23  GPS corrections =  0
OA BT24  No sensors !
OA BT24  No digitizers !
OA BT24  GPS corrections =  0
OA BT25  GPS corrections =  0
OA BT26  No sensors !
OA BT26  No digitizers !
OA BT26  GPS corrections =  0
OA BT27  No sensors !
OA BT27  No digitizers !
OA BT27  GPS corrections =  0
OA BT28  No sensors !
OA BT28  No digitizers !
OA BT28  GPS corrections =  0
OA BU22  GPS corrections =  0
OA BU23  GPS corrections =  0
OA BU24  GPS corrections =  0
OA BU25  GPS corrections =  0
OA BU26  GPS corrections =  0
OA BU27  GPS corrections =  0
OA BU28  No sensors !
OA BU28  No digitizers !
OA BU28  GPS correcti

OA BU22  GPS corrections =  0
OA BU23  GPS corrections =  0
OA BU24  GPS corrections =  0
OA BU25  GPS corrections =  0
OA BU26  GPS corrections =  0
OA BU27  GPS corrections =  0
OA BV21  GPS corrections =  0
OA BV22  GPS corrections =  0
OA BV23  GPS corrections =  0
OA BV24  GPS corrections =  0
OA BV26  GPS corrections =  0
OA BV27  No digitizers !
OA BV27  GPS corrections =  0
OA BW20  GPS corrections =  0
OA BW21  GPS corrections =  0
OA BW22  GPS corrections =  0
OA BW23  GPS corrections =  0
OA BW24  GPS corrections =  0
OA BW25  No sensors !
OA BW25  No digitizers !
OA BW25  GPS corrections =  0
OA BW26  No sensors !
OA BW26  No digitizers !
OA BW26  GPS corrections =  0
OA BW27  GPS corrections =  0
OA BX21  GPS corrections =  0
OA BX22  GPS corrections =  0
OA BX23  GPS corrections =  0
OA BX24  GPS corrections =  0
OA BX25  GPS corrections =  0
OA BX26  GPS corrections =  0
OA BX27  GPS corrections =  0
OA BX28  GPS corrections =  0
OA BY20  GPS corrections =  0
OA BY21  GP

In [15]:
Stop here to check output .

SyntaxError: invalid syntax (<ipython-input-15-d10a2bbcc13e>, line 1)

# Test how to equipment into station nodes 

In [ ]:
from extract_equipments_from_csv import  EquipmentExtractor
import json

my_equip_obj = EquipmentExtractor()

for a_net in big_inv.networks:
    
    print("The number of station-nodes in the network =", len(a_net.stations))
    
    for a_sta in a_net.stations:
        # print(a_net.code, a_sta.code)  # this contains 328 pairs, but they are NOT unique, station code may repeat.

        
        a_inv = big_inv.select(network=a_net.code,station=a_sta.code).copy()  # .copy appears to have no effect here
        
        # print (a_sta.code, " stations has %s channels"%len(a_sta))
        
        _sensors = my_equip_obj.get_sensors(a_net.code, a_sta.code)
        if len(_sensors)>0:
            sensor_desc =_sensors[0].get("Description")
            sensor_sernumb =_sensors[0].get("SerNumber")
        else:
            print("Warning: No sensors for (%s,%s)"%(a_net.code, a_sta.code))
            sensor_desc = "NA Sensor for (%s,%s)"%(a_net.code, a_sta.code)
            sensor_sernumb = "NA"
            
        _digitizers = my_equip_obj.get_digitizer(a_net.code, a_sta.code)
        if len(_digitizers)>0:
            dig_desc =_digitizers[0].get("Description")
            dig_sernumb = _digitizers[0].get("SerNumber")
        else:
            print("Warning: No digitizer for (%s,%s)"%(a_net.code, a_sta.code))
            dig_desc = "NA Digitizer for (%s,%s)"%(a_net.code, a_sta.code)
            dig_sernumb = "NA"
         
        
        # modify station metadata
        my_sensor=obspy.core.inventory.util.Equipment(type="Sensor", description=sensor_desc,serial_number=sensor_sernumb)
        
        #my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description="Guralp Minimus",serial_number="MIN-A456")
        my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description=dig_desc,serial_number=dig_sernumb)

        a_sta.equipments = [my_sensor, my_digitizer]
              
#         sta_file_name2 = "%s_%s_station2.xml"%(a_net.code, a_sta.code)  
#         # OA_CE28 was written 3-times!!!!!! due to multiple (OA,CE28)-station-nodes 
#         There will be 119 xml files written in this loop of 328 items. However, the final results missed 119 equipments!!
#         outxml2 = os.path.join(OUTPUT_DIR, sta_file_name2)

#         inv2.networks = a_inv.networks

#         inv2.write(outxml2,format="stationxml", validate=True) # nsmap={'GeoscienceAustralia': GA_NameSpace})

    # the big inventory in new object inv2 writing out -- appears OK 328 Sensor and Digitizer
    inv2.networks = []  
    inv2.networks.append(a_net)
    inv2.write(a_net.code+"_stations2.xml", format="stationxml", validate=True)  # every Station got equipment
    
    # The original write out again now:
    
    big_inv.write(a_net.code+"_stations_post.xml", format="stationxml", validate=True) # also has the Sensors etc
    

# TODOs:  

## for each (net, sta):
* get the real Sensor and Ditizer from OA_sensors_digitizers.csv 

* get the GPS corrections

* get the orientation corrections



###

(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ grep "Guralp Minimus"  OA_sensors_digitizers.csv | wc
    262     524   10856
(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ grep "120s"  OA_sensors_digitizers.csv | wc
    218     872   12654
(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ wc OA_sensors_digitizers.csv
  481  1397 23683 OA_sensors_digitizers.csv
(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ grep BV26 OA_sensors_digitizers.csv
33850940,OABV26,,Guralp Minimus,MIN-4556
33850940,OABV26,,Nanometrics Trillium Compact 120s,004768


In [ ]:
# inv2.write(a_net.code+"_stations3.xml", format="stationxml", nsmap={'GeoscienceAustralia': GA_NameSpace}, validate=True)  # every Station got equipment
    

# How to merge the individual xml files into One?

## use new obspy version

In [ ]:
stop here

# How to create a new station xml file

Adapted from https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html


In [ ]:
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL


def create_new_inventory(out_file_name="new_inventory.xml"):

    # We'll first create all the various objects. These strongly follow the
    # hierarchy of StationXML files.
    inv = Inventory(
        # We'll add networks later.
        networks=[],
        # The source should be the id whoever create the file.
        source="Geoscience Australia EFTF Program AusArray")

    net = Network(
        # This is the network code according to the SEED standard.
        code="XX",
        # A list of stations. We'll add one later.
        stations=[],
        description="A test stations.",
        # Start-and end dates are optional.
        start_date=obspy.UTCDateTime(2016, 1, 2))

    sta = Station(
        # This is the station code according to the SEED standard.
        code="ABC",
        latitude=1.0,
        longitude=2.0,
        elevation=345.0,

        creation_date=obspy.UTCDateTime(2016, 1, 2), # <CreationDate>2016-01-02T00:00:00.000000Z</CreationDate>

        start_date = obspy.UTCDateTime(2016, 1, 3),
        end_date = obspy.UTCDateTime(2020, 1, 3),
        #<Station code="ABC" startDate="2016-01-03T00:00:00.000000Z" endDate="2020-01-03T00:00:00.000000Z">
        site=Site(name="First station"))

    cha = Channel(
        # This is the channel code according to the SEED standard.
        code="HHZ",
        # This is the location code according to the SEED standard.
        location_code="",
        # Note that these coordinates can differ from the station coordinates.
        latitude=1.0,
        longitude=2.0,
        elevation=345.0,
        depth=10.0,
        azimuth=0.0,
        dip=-90.0,
        sample_rate=200)

    # By default this accesses the NRL online. Offline copies of the NRL can
    # also be used instead
    nrl = NRL()
    # The contents of the NRL can be explored interactively in a Python prompt,
    # see API documentation of NRL submodule:
    # http://docs.obspy.org/packages/obspy.clients.nrl.html
    # Here we assume that the end point of data logger and sensor are already
    # known:
    response = nrl.get_response( # doctest: +SKIP
        sensor_keys=['Streckeisen', 'STS-1', '360 seconds'],
        datalogger_keys=['REF TEK', 'RT 130 & 130-SMA', '1', '200'])


    # Now tie it all together.
    cha.response = response
    sta.channels.append(cha)
    net.stations.append(sta)
    inv.networks.append(net)

    # And finally write it to a StationXML file. We also force a validation against
    # the StationXML schema to ensure it produces a valid StationXML file.
    #
    # Note that it is also possible to serialize to any of the other inventory
    # output formats ObsPy supports.
    inv.write(out_file_name, format="stationxml", validate=True)

In [ ]:
create_new_inventory()